# Zadanie domowe

W przypadku obrazów w odcieniach szarości pojedynczy piksel z zakresu [0; 255] reprezentowany jest jako 8-bitowa liczba bez znaku.
Pewnym rozszerzeniem analizy sposobu reprezentacji obrazu może być następujący eksperyment.
Załóżmy, że z każdego z 8 bitów możemy stworzyć pojedynczy obraz binarny (ang. _bit-plane slicing_).
Dla obrazka _100zloty.jpg_ (https://raw.githubusercontent.com/vision-agh/poc_sw/master/02_Point/100zloty.jpg) stwórz 8 obrazów, z których każdy powinien zawierać jedną płaszczyznę bitową.
Podpowiedź $-$ warto sprawdzić, jak realizuje się bitowe operacje logiczne.
Zastosowanie takiej dekompozycji obrazu pozwala na analizę ,,ważności'' poszczególnych bitów.
Jest to użyteczne w kwantyzacji, ale także w kompresji.

W drugim etapie zadania proszę spróbować odtworzyć obraz oryginalny z mniejszej liczby obrazów binarnych.
Warto zacząć od dwóch najbardziej znaczących bitów, a później dodawać kolejne.
Należy utworzyć co najmniej trzy wersje zrekonstruowanych obrazów.
Podpowiedź $-$ rekonstrukcja obrazu to mnożenie przez odpowiednią potęgę liczby 2 (przesunięcie bitowe) oraz dodawanie.

In [1]:
import cv2
import os
import requests
import matplotlib.pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/02_Point/'
fileName = '100zloty.jpg'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
img = cv2.imread(fileName, cv2.IMREAD_GRAYSCALE)

bit_planes = []
for i in range(8):
    bit_plane = (img >> i) & 1
    bit_plane = (bit_plane * 255).astype(np.uint8)
    bit_planes.append(bit_plane)

plt.figure(figsize=(12, 6))
for i in range(8):
    plt.subplot(2, 4, i + 1)
    plt.imshow(bit_planes[i], cmap='gray')
    plt.title(f'Bit plane {i}')
    plt.axis('off')
plt.suptitle('Płaszczyzny bitowe (od najmniej do najbardziej znaczącego bitu)')
plt.tight_layout()
plt.show()


recon_2bits = ((bit_planes[7] // 255) << 7) + ((bit_planes[6] // 255) << 6)

recon_4bits = sum(((bit_planes[i] // 255) << i) for i in range(4, 8))

recon_6bits = sum(((bit_planes[i] // 255) << i) for i in range(2, 8))

plt.figure(figsize=(12, 4))
titles = ['Oryginalny obraz', '2 bity (6-7)', '4 bity (4-7)', '6 bitów (2-7)']
images = [img, recon_2bits, recon_4bits, recon_6bits]

for i in range(4):
    plt.subplot(1, 4, i + 1)
    plt.imshow(images[i], cmap='gray')
    plt.title(titles[i])
    plt.axis('off')

plt.suptitle('Rekonstrukcja obrazu z różnych liczby płaszczyzn bitowych')
plt.tight_layout()
plt.show()

In [ ]:
img_bgr = cv2.imread(fileName)

def construct_color_image(r_planes, g_planes, b_planes, start_bit=0, end_bit=7):
    r = sum(((r_planes[i] // 255) << i) for i in range(start_bit, end_bit + 1))
    g = sum(((g_planes[i] // 255) << i) for i in range(start_bit, end_bit + 1))
    b = sum(((b_planes[i] // 255) << i) for i in range(start_bit, end_bit + 1))
    return np.stack([r, g, b], axis=-1).astype(np.uint8)

def bit_planes_for_channel(channel):
    planes = []
    for i in range(8):
        plane = (channel >> i) & 1
        plane = (plane * 255).astype(np.uint8)
        planes.append(plane)
    return planes

b_planes = bit_planes_for_channel(img_rgb[:, :, 0])  # Blue
g_planes = bit_planes_for_channel(img_rgb[:, :, 1])  # Green
r_planes = bit_planes_for_channel(img_rgb[:, :, 2])  # Red

binaries = []
num_bits = 8
for i in range(num_bits):
    binary = (img_bgr >> i) & 1
    binary = (binary * 255).astype(np.uint8)
    binaries.append(binary)

fig, axes = plt.subplots(8, 1, figsize=(32, 16))
for i in range(8):
    axes[i].imshow(binaries[i])
    axes[i].set_title(f'Bit {i}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

recon_2bits = construct_color_image(r_planes, g_planes, b_planes, 6, 7)
recon_4bits = construct_color_image(r_planes, g_planes, b_planes, 4, 7)
recon_6bits = construct_color_image(r_planes, g_planes, b_planes, 2, 7)

plt.figure(figsize=(12, 4))
titles = ['Oryginalny obraz', '2 bity (6–7)', '4 bity (4–7)', '6 bitów (2–7)']
images = [img_rgb, recon_2bits, recon_4bits, recon_6bits]

for i in range(4):
    plt.subplot(1, 4, i + 1)
    plt.imshow(images[i])
    plt.title(titles[i])
    plt.axis('off')

plt.suptitle('Rekonstrukcja kolorowego obrazu z różnych liczby płaszczyzn bitowych')
plt.tight_layout()
plt.show()